In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR
from kaggle.competitions import twosigmanews

In [ ]:
env = twosigmanews.make_env()

TODO: news_train_df も利用する

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
market_train_df.describe()

In [ ]:
train_df = market_train_df.loc[:,["returnsClosePrevRaw1", "returnsOpenPrevRaw1", "returnsClosePrevMktres1", "returnsOpenPrevMktres1", "returnsClosePrevRaw10", "returnsOpenPrevRaw10", "returnsClosePrevMktres10", "returnsOpenPrevMktres10"]]

In [ ]:
target_df = market_train_df.loc[:,["returnsOpenNextMktres10"]]

In [ ]:
values = target_df.values

TODO: この分散を正規化していきたい
* Logだとマイナスの値が使えない
* 大きい数の外れ値があるけど、この外れ値は収入が大きかったときだからしっかり使いたい

In [ ]:
plt.hist(values) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target_df)

In [ ]:
lgb_train = lgb.Dataset(train_df, target_df)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

TODO: ハイパーパラメータ設定

In [ ]:
lgbm_params = {}

In [ ]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

In [ ]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
plt.hist(y_pred)

In [ ]:
y_pred[y_pred<-1].size

In [ ]:
y_pred[y_pred>1].size

TODO: confidenceValueのロジックを考える

In [ ]:
y_pred[y_pred > 1.0] = 1.0
y_pred[y_pred < -1.0] = -1.0

In [ ]:
days = env.get_prediction_days()

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    market_df = market_obs_df.loc[:,["returnsClosePrevRaw1", "returnsOpenPrevRaw1", "returnsClosePrevMktres1", "returnsOpenPrevMktres1", "returnsClosePrevRaw10", "returnsOpenPrevRaw10", "returnsClosePrevMktres10", "returnsOpenPrevMktres10"]]
    y_pred = model.predict(market_df, num_iteration=model.best_iteration)
    y_pred[y_pred > 1.0] = 1.0
    y_pred[y_pred < -1.0] = -1.0
    predictions_template_df.confidenceValue = y_pred
    env.predict(predictions_template_df)
print('Done!')

In [ ]:
env.write_submission_file()

In [ ]:
import os
print([filename for filename in os.listdir('.') if '.csv' in filename])